# Grad Cafe Data Analysis

## Introduction

In [102]:
import re
import time
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
from IPython.display import Image

In [103]:
DATA_PATH = 'data'

## Scraping

In this section, we will use `requests` to crawl the data from Grad Cafe.

Admission results can be queried at [https://thegradcafe.com/survey/index.php](https://thegradcafe.com/survey/index.php). Grad Cafe doesn't support browsing through the whole database. Instead, at least one keyword must be used to retrieve the results. Since we are interested in CS programs, we will use the keyword `Computer` which returns 192 pages of results with 250 results per page.

Grad Cafe organizes the results in a `table`, which is not very difficult to parse. Therefore, we can simply retrieve the HTML source files and save them for later parsing.

Note that Grad Cafe forbids requests without a proper `User-agent` header. Apart from that, there seems to be no anti-crawler mechanisms.

In [116]:
for pid in range(1, 200):
    url = 'https://thegradcafe.com/survey/index.php?q=engineering&t=a&pp=250&o=&p={0}'.format(pid)
    resp = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
    if resp.status_code != 200:
        print('Failed at', pid)
        break
    with open('{0}/{1}.html'.format(DATA_PATH, pid), 'w') as f:
        f.write(resp.text)
    time.sleep(1)

## Parsing

In this section, we will parse the HTML files using `BeautifulSoup` and regular expressions. The goal is to convert them into a Pandas DataFrame.

The image above shows what is contained in an admission result. From this we can derive what columns our DataFrame will contain:

- `university`: name of the university.
- `major`: name of the major.
- `degree`: can be one of `PhD`, `Masters`, `MFA`, `MBA`, `JD`, `EdD`, `Other`.
- `season`: season of admission, F/S (Fall/Spring) + year.
- `decision`: admission decision, can be one of `Accepted`, `Rejected`, `Interview`, `Wait listed`, `Other`.
- `decision_method`: method of decision notification, can be one of `Postal Service`, `E-mail`, `Phone`, `Website`, `Other`.
- `decision_date`: date of decision notification.
- `gpa`: undergraduate GPA.
- `gre_verbal`: GRE verbal score.
- `gre_quant`: GRE quantitative score.
- `gre_writing`: GRE analytical writing score.
- `status`: indicates American/international student status.
- `created_at`: date that this result was added.
- `comment`: any notes the user has made.

Note that we will not extract GRE subject score, as CS programs generally do not require a GRE subject score.

In [117]:
column_names = [
    'university',
    'major',
    'degree',
    'season',
    'decision',
    'decision_method',
    'decision_date',
    'gpa',
    'gre_verbal',
    'gre_quant',
    'gre_writing',
    'status',
    'created_at',
    'comment'
]

Some fields are in text format and we need to define several regular expressions to parse them.

In [118]:
pat_decision_info = re.compile((
'((?:Accepted)|(?:Rejected)|(?:Interview)|(?:Wait listed)|(?:Other)) via '
'((?:Postal Service)|(?:E-mail)|(?:Phone)|(?:Website)|(?:Other))'
))
pat_degree = re.compile('((?:PhD)|(?:Masters)|(?:MFA)|(?:MBA)|(?:JD)|(?:EdD)|(?:Other))')
pat_season = re.compile('(\([SF][01][0-9]\))')
pat_gpa = re.compile('GPA: ((?:[0-9]\.[0-9]{1,2})|(?:n/a))')
pat_gre = re.compile('GRE General \(V/Q/W\): ([0-9]{2,3})/([0-9]{2,3})/([0-9]\.[0-9]{1,2})')

Next, let's write a function that will parse one row (one result) from the results table. This is a regualr expression heavy process and involves a lot of details. Please refer to the comments in the code.

In [119]:
def parse_row(row):
    cols = row.findAll('td')
    
    university = cols[0].text
    
    # '(Computer Science ) MS Thesis, Masters (F18)'
    program_season_info = cols[1].text
    pos = program_season_info.rfind(',')
    # '(Computer Science ) MS Thesis'
    major = program_season_info[:pos].strip()
    # 'Masters (F18)'
    degree_season_info = program_season_info[pos+1:].strip()
    ret_degree = pat_degree.search(degree_season_info)
    # 'Masters'
    degree = ret_degree.groups()[0] if ret_degree else None
    ret_season = pat_season.search(degree_season_info)
    # 'F18'
    season = ret_season.groups()[0][1:-1] if ret_season else None
    
    # 'Rejected via E-mail on 25 Apr 2018 ...'
    decision_info = cols[2].text
    # GPA and GRE scores are inside the child elements, which will also be captured by `.text` attribute
    # Need to slice that out
    pos = cols[2].text.find('Undergrad')
    if pos != -1:
        decision_info = decision_info[:pos]
    ret_decision_info = pat_decision_info.search(decision_info)
    # ('Rejected', 'E-Mail')
    decision, decision_method = ret_decision_info.groups() if ret_decision_info else (None, None)
    pos = decision_info.rfind('on')
    # '25 Apr 2018'
    decision_date = decision_info[pos+2:].strip()
    if not decision_date: decision_date = None
    
    score_info_tag = cols[2].find(class_='extinfo')
    if score_info_tag:
        # 'Undergrad GPA: 3.90 GRE General (V/Q/W): 164/170/4.00 ...'
        score_info = score_info_tag.text
        ret_gpa = pat_gpa.search(score_info)
        # '3.90'
        gpa = ret_gpa.groups()[0] if ret_gpa else None
        if gpa == 'n/a': gpa = None
        ret_gre = pat_gre.search(score_info)
        # ('164', '170', '4.00')
        gre_verbal, gre_quant, gre_writing = ret_gre.groups() if ret_gre else (None, None, None)
    else:
        gpa, gre_verbal, gre_quant, gre_writing = None, None, None, None
    
    # 'I'
    status = cols[3].text.strip()
    if not status: status = None
    
    # '25 Apr 2018'
    created_at = cols[4].text.strip()
    if not created_at: created_at = None
    
    # 'Large number of strong applicants it seems. ...'
    comment = cols[5].text.strip()
    if not comment: comment = None
    
    return [university, major, degree, season, decision, decision_method, decision_date, gpa, gre_verbal, gre_quant, gre_writing, status, created_at, comment]

Now let's write a function that finds all the rows from a HTML file and calls `parse_row` to obtain all the parsed data from it.

In [120]:
from tqdm.auto import tqdm

In [121]:
def parse_file(filename):
    with open(filename) as f:
        html = f.read()
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    # Skip the header
    rows = table.findAll('tr')[1:]
    results = []
    for idx, row in enumerate(rows):
        results.append(parse_row(row))
    return results

Now we're ready to parse all the HTML files we obtained by web scraping.

In [125]:
data = []
for pid in tqdm(range(1, 200)):
    filename = '{0}/{1}.html'.format(DATA_PATH, pid)
    data.extend(parse_file(filename))

  0%|          | 0/199 [00:00<?, ?it/s]

Finally, we are able to create a Pandas DataFrame with the parsed data, which has in total 47745 rows. A quicks look into the DataFrame shows that the results is pretty desirable.

In [126]:
df = pd.DataFrame(data)
df.columns = column_names
print(len(df))
df.head(5)

49750


,university,major,degree,season,decision,decision_method,decision_date,gpa,gre_verbal,gre_quant,gre_writing,status,created_at,comment
0,Sri Manakula Vinayagar Engineering College,Computer Science,Masters,None,Accepted,E-mail,20 Nov 2021,3.20,140,149,2.50,I,20 Nov 2021,report spam\nreply
1,Michigan State University,Mechanical Engineering,PhD,None,Rejected,Website,19 Nov 2021,3.63,None,None,None,I,19 Nov 2021,I am rejecting without mentioning any reason I...
2,Johns Hopkins University,Mechanical Engineering,PhD,None,Rejected,E-mail,17 Nov 2021,None,None,None,None,U,17 Nov 2021,Department resubmitted my application for Fall...
3,Stevens Institute Of Technology,Mechanical Engineering,PhD,None,Rejected,E-mail,16 Nov 2021,3.85,156,161,4.00,I,16 Nov 2021,"""...We have reviewed your application for a Ph..."
4,UIUC,Civil And Environmental Engineering,Masters,None,Accepted,E-mail,5 May 2021,3.42,155,162,3.50,I,14 Nov 2021,Applied for PhD but accepted for non thesis ma...


In [127]:
df

,university,major,degree,season,decision,decision_method,decision_date,gpa,gre_verbal,gre_quant,gre_writing,status,created_at,comment
0,Sri Manakula Vinayagar Engineering College,Computer Science,Masters,None,Accepted,E-mail,20 Nov 2021,3.20,140,149,2.50,I,20 Nov 2021,report spam\nreply
1,Michigan State University,Mechanical Engineering,PhD,None,Rejected,Website,19 Nov 2021,3.63,None,None,None,I,19 Nov 2021,I am rejecting without mentioning any reason I...
2,Johns Hopkins University,Mechanical Engineering,PhD,None,Rejected,E-mail,17 Nov 2021,None,None,None,None,U,17 Nov 2021,Department resubmitted my application for Fall...
3,Stevens Institute Of Technology,Mechanical Engineering,PhD,None,Rejected,E-mail,16 Nov 2021,3.85,156,161,4.00,I,16 Nov 2021,"""...We have reviewed your application for a Ph..."
4,UIUC,Civil And Environmental Engineering,Masters,None,Accepted,E-mail,5 May 2021,3.42,155,162,3.50,I,14 Nov 2021,Applied for PhD but accepted for non thesis ma...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49745,Clemson University,Computer Engineering,PhD,F15,Rejected,E-mail,29 Jan 2015,3.86,147,167,3.50,I,29 Jan 2015,report spam\nreply
49746,"University Of California, San Diego",Computer Science And Engineering,Masters,F15,Accepted,E-mail,29 Jan 2015,None,170,154,3.00,I,29 Jan 2015,Non-CS undergrad. One paper in preparation. On...
49747,University Of Texas Austin,Mechanical Engineering,PhD,F15,Rejected,Website,28 Jan 2015,None,None,None,None,I,29 Jan 2015,"1 Journal Publication,and another close to Pub..."
49748,NC State,Nuclear Engineering,PhD,F15,Accepted,E-mail,29 Jan 2015,3.64,162,164,4.50,A,29 Jan 2015,report spam\nreply


In [128]:
df.to_csv("gradcafe.csv")